In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


from sklearn.model_selection import train_test_split

df = pd.read_csv('srednjeni_podaci.csv', parse_dates=True)
df.head

<bound method NDFrame.head of      Unnamed: 0  godina       jan       feb        mar        apr        maj  \
0             0  1887.0  0.295349  2.032558   6.925581  12.193798  17.117054   
1             1  1888.0 -6.500000 -2.300000   7.100000  10.900000  15.900000   
2             2  1889.0 -4.000000 -0.900000   3.700000  11.000000  18.200000   
3             3  1890.0  1.300000 -1.700000   6.500000  13.000000  17.600000   
4             4  1891.0 -6.400000 -4.800000   6.500000   9.400000  19.100000   
..          ...     ...       ...       ...        ...        ...        ...   
126         126  2013.0  3.200000  4.400000   6.600000  14.800000  18.900000   
127         127  2014.0  5.200000  7.600000  10.800000  13.600000  16.900000   
128         128  2015.0  3.800000  4.000000   8.100000  13.200000  18.800000   
129         129  2016.0  2.400000  9.000000   9.000000  15.400000  17.300000   
130         130  2017.0  0.295349  2.032558   6.925581  12.193798  17.117054   

         

In [2]:
df.columns

Index(['Unnamed: 0', 'godina', 'jan', 'feb', 'mar', 'apr', 'maj', 'jun', 'jul',
       'avg', 'sep', 'okt', 'nov', 'dec', 'prosek', 'zima', 'proleće', 'leto',
       'jesen', 'tsr-amp-mes', 'tsr leto-zima'],
      dtype='object')

In [3]:
df = df.drop(df.columns[0], axis=1)

In [4]:

df['godina'] = pd.to_datetime(df['godina'], format='%Y')


df.set_index('godina', inplace=True)

train_data, test_data = train_test_split(df['prosek'], test_size=0.2, shuffle=False)

# Pronalaženje optimalnih parametara pomoću auto_arima
model = auto_arima(train_data, seasonal=True, m=12, suppress_warnings=True)


sarima_model = SARIMAX(train_data, order=model.order, seasonal_order=model.seasonal_order)
result = sarima_model.fit()

# Predviđanje na test skupu
forecast = result.get_forecast(steps=len(test_data))
predicted_mean = forecast.predicted_mean


mse = mean_squared_error(test_data, predicted_mean)
mae = mean_absolute_error(test_data, predicted_mean)
print("Srednja kvadratna greška (MSE):", mse)
print("Srednja apsolutna greška (MAE):", mae)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.16396D+00    |proj g|=  9.36186D-02

At iterate    5    f=  1.11542D+00    |proj g|=  5.03277D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      9     12      1     0     0   6.168D-08   1.115D+00
  F =   1.1152461039236039     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
Srednja kvadratna greška (MSE): 1.6036087998820945
Srednja apsolutna greška (MAE): 1.0704202729101386


/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


In [5]:
# Dobijanje predviđanja za buduće periode
future_steps = 5  
forecast = result.get_forecast(steps=future_steps)


predviđene_vrijednosti = forecast.predicted_mean
intervali_pouzdanosti = forecast.conf_int()

# Ispisivanje predviđenih vrednosti i intervala pouzdanosti
print("Predviđene vrijednosti:")
print(predviđene_vrijednosti)
print("\nIntervali pouzdanosti:")
print(intervali_pouzdanosti)

Predviđene vrijednosti:
1991-01-01    11.96794
1992-01-01    11.96794
1993-01-01    11.96794
1994-01-01    11.96794
1995-01-01    11.96794
Freq: AS-JAN, Name: predicted_mean, dtype: float64

Intervali pouzdanosti:
            lower prosek  upper prosek
1991-01-01     10.518226     13.417654
1992-01-01     10.512711     13.423170
1993-01-01     10.507217     13.428664
1994-01-01     10.501743     13.434138
1995-01-01     10.496289     13.439591


In [6]:
train_data, test_data = train_test_split(df['leto'], test_size=0.2, shuffle=False)

best_mse = float('inf')
best_m = None

# Eksperimentisanje sa različitim vrednostima za m
for m in [4, 6, 12]:
    # Pronalaženje optimalnih parametara pomoću auto_arima
    model_summer = auto_arima(train_data, seasonal=True, m=m, suppress_warnings=True)
    # Treniranje SARIMA modela sa optimalnim parametrima
    sarima_model_summer = SARIMAX(train_data, order=model_summer.order, seasonal_order=model_summer.seasonal_order)
    result = sarima_model_summer.fit()

    # Predviđanje na test skupu
    forecast = result.get_forecast(steps=len(test_data))
    predicted_mean = forecast.predicted_mean

    # Računanje srednje kvadratne greške
    mse = mean_squared_error(test_data, predicted_mean)

    print("Srednja kvadratna greška (MSE) za m =", m, ":", mse)

    # Ažuriranje najbolje vrednosti MSE i vrednosti m
    if mse < best_mse:
        best_mse = mse
        best_m = m

print("Najbolji m:", best_m)
print("Najbolja srednja kvadratna greška (MSE):", best_mse)

/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.17329D+00    |proj g|=  3.81070D-01

At iterate    5    f=  3.36904D+00    |proj g|=  1.04495D+00

At iterate   10    f=  2.91451D+00    |proj g|=  5.78583D-02

At iterate   15    f=  2.84210D+00    |proj g|=  3.16836D-02

At iterate   20    f=  2.83203D+00    |proj g|=  8.09367D-04

At iterate   25    f=  2.83133D+00    |proj g|=  7.19410D-04

At iterate   30    f=  2.83126D+00    |proj g|=  1.14360D-04

At iterate   35    f=  2.83120D+00    |proj g|=  7.50874D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function 

/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.49445D+00    |proj g|=  2.33027D+00

At iterate    5    f=  1.48711D+00    |proj g|=  2.06303D-02

At iterate   10    f=  1.48561D+00    |proj g|=  1.33365D-03

At iterate   15    f=  1.48560D+00    |proj g|=  7.93574D-04

At iterate   20    f=  1.48556D+00    |proj g|=  2.76189D-04

At iterate   25    f=  1.48555D+00    |proj g|=  1.82697D-03

At iterate   30    f=  1.48500D+00    |proj g|=  2.52917D-02

At iterate   35    f=  1.48011D+00    |proj g|=  7.40088D-03

At iterate   40    f=  1.47900D+00    |proj g|=  3.46155D-03

At iterate   45    f=  1.47894D+00    |proj g|=  7.20229D-04

At iterate   50    f=  1.47893D+00    |proj g|=  8.40382D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/home/marija/.local/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.44616D+00    |proj g|=  2.11590D+00

At iterate    5    f=  1.48904D+00    |proj g|=  3.57595D-02

At iterate   10    f=  1.48530D+00    |proj g|=  9.73079D-03

At iterate   15    f=  1.48495D+00    |proj g|=  7.27165D-03

At iterate   20    f=  1.48149D+00    |proj g|=  2.23663D-02

At iterate   25    f=  1.47965D+00    |proj g|=  3.40605D-03

At iterate   30    f=  1.47964D+00    |proj g|=  7.14532D-04

At iterate   35    f=  1.47946D+00    |proj g|=  1.03541D-02

At iterate   40    f=  1.47440D+00    |proj g|=  2.59274D-02

At iterate   45    f=  1.47172D+00    |proj g|=  1.84125D-02

At iterate   50    f=  1.47084D+00    |proj g|=  2.09773D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/home/marija/.local/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [7]:
train_data, test_data = train_test_split(df['leto'], test_size=0.2, shuffle=False)

# Pronalaženje optimalnih parametara pomoću auto_arima
model_summer = auto_arima(train_data, seasonal=True, m=12 , suppress_warnings=True)
# Treniranje SARIMA modela sa optimalnim parametrima
sarima_model_summer = SARIMAX(train_data, order=model_summer.order, seasonal_order=model_summer.seasonal_order)
result_summer = sarima_model_summer.fit()

# Predviđanje na test skupu
forecast = result_summer.get_forecast(steps=len(test_data))
predicted_mean = forecast.predicted_mean

# Računanje srednje kvadratne greške
mse = mean_squared_error(test_data, predicted_mean)
mae = mean_absolute_error(test_data, predicted_mean)
print("Srednja kvadratna greška summer(MSE):", mse)
print("Srednja apsolutna greška summer (MAE):", mae)

/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.44616D+00    |proj g|=  2.11590D+00

At iterate    5    f=  1.48904D+00    |proj g|=  3.57595D-02

At iterate   10    f=  1.48530D+00    |proj g|=  9.73079D-03

At iterate   15    f=  1.48495D+00    |proj g|=  7.27165D-03

At iterate   20    f=  1.48149D+00    |proj g|=  2.23663D-02

At iterate   25    f=  1.47965D+00    |proj g|=  3.40605D-03

At iterate   30    f=  1.47964D+00    |proj g|=  7.14532D-04

At iterate   35    f=  1.47946D+00    |proj g|=  1.03541D-02

At iterate   40    f=  1.47440D+00    |proj g|=  2.59274D-02

At iterate   45    f=  1.47172D+00    |proj g|=  1.84125D-02

At iterate   50    f=  1.47084D+00    |proj g|=  2.09773D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/home/marija/.local/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [8]:
future_steps = 5 
forecast = result.get_forecast(steps=future_steps)


predviđene_vrijednosti = forecast.predicted_mean
intervali_pouzdanosti = forecast.conf_int()


print("Predviđene vrijednosti:")
print(predviđene_vrijednosti)
print("\nIntervali pouzdanosti:")
print(intervali_pouzdanosti)

Predviđene vrijednosti:
1991-01-01    21.382505
1992-01-01    21.273872
1993-01-01    21.189805
1994-01-01    21.094997
1995-01-01    21.108494
Freq: AS-JAN, Name: predicted_mean, dtype: float64

Intervali pouzdanosti:
            lower leto  upper leto
1991-01-01   19.394741   23.370269
1992-01-01   19.145353   23.402391
1993-01-01   19.052815   23.326795
1994-01-01   18.948039   23.241956
1995-01-01   18.951660   23.265329


In [9]:
train_data, test_data = train_test_split(df['zima'], test_size=0.2, shuffle=False)

best_mse = float('inf')
best_m = None

for m in [4, 6, 12]:
   
    model_winter = auto_arima(train_data, seasonal=True, m=m, suppress_warnings=True)
    
    sarima_model_winter = SARIMAX(train_data, order=model_winter.order, seasonal_order=model_winter.seasonal_order)
    result_winter = sarima_model_winter.fit()

    forecast = result_winter.get_forecast(steps=len(test_data))
    predicted_mean = forecast.predicted_mean

    mse = mean_squared_error(test_data, predicted_mean)

    print("Srednja kvadratna greška (MSE) za m =", m, ":", mse)

  
    if mse < best_mse:
        best_mse = mse
        best_m = m

print("Najbolji m:", best_m)
print("Najbolja srednja kvadratna greška (MSE)-winter:", best_mse)

/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.17308D+00    |proj g|=  1.84872D-01

At iterate    5    f=  2.14127D+00    |proj g|=  6.43524D-03

At iterate   10    f=  2.13844D+00    |proj g|=  2.24108D-02

At iterate   15    f=  2.13724D+00    |proj g|=  2.92852D-03

At iterate   20    f=  2.13718D+00    |proj g|=  7.81006D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     20     22      1     0     0   7.810D-06   2.137D+00
  F =   2.1371758289265430     

CONVERG

/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.18650D+00    |proj g|=  6.23007D-02

At iterate    5    f=  2.15513D+00    |proj g|=  2.05252D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      8     10      1     0     0   2.168D-07   2.155D+00
  F =   2.1550724125353686     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
Srednja kvadratna greška (MSE) za m = 6 : 3.2347622024085125
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D

/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


In [10]:

train_data, test_data = train_test_split(df['zima'], test_size=0.2, shuffle=False)


model_winter = auto_arima(train_data, seasonal=True, m=4 , suppress_warnings=True)

sarima_model_winter = SARIMAX(train_data, order=model_winter.order, seasonal_order=model_winter.seasonal_order)
result_winter = sarima_model_winter.fit()

forecast = result_winter.get_forecast(steps=len(test_data))
predicted_mean = forecast.predicted_mean

mse = mean_squared_error(test_data, predicted_mean)
mae = mean_absolute_error(test_data, predicted_mean)
print("Srednja kvadratna greška (MSE)-winter:", mse)
print("Srednja apsolutna greška (MAE)-winter:", mae)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.17308D+00    |proj g|=  1.84872D-01

At iterate    5    f=  2.14127D+00    |proj g|=  6.43524D-03

At iterate   10    f=  2.13844D+00    |proj g|=  2.24108D-02

At iterate   15    f=  2.13724D+00    |proj g|=  2.92852D-03

At iterate   20    f=  2.13718D+00    |proj g|=  7.81006D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     20     22      1     0     0   7.810D-06   2.137D+00
  F =   2.1371758289265430     

CONVERG

/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/home/marija/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


In [11]:
future_steps = 5  
forecast = result_winter.get_forecast(steps=future_steps)


predviđene_vrijednosti = forecast.predicted_mean
intervali_pouzdanosti = forecast.conf_int()

print("Predviđene vrijednosti- zima:")
print(predviđene_vrijednosti)
print("\nIntervali pouzdanosti- zima:")
print(intervali_pouzdanosti)

Predviđene vrijednosti- zima:
1991-01-01    2.035877
1992-01-01    2.049530
1993-01-01    2.259466
1994-01-01    1.941178
1995-01-01    2.237509
Freq: AS-JAN, Name: predicted_mean, dtype: float64

Intervali pouzdanosti- zima:
            lower zima  upper zima
1991-01-01   -2.030261    6.102016
1992-01-01   -2.017478    6.116537
1993-01-01   -1.884593    6.403526
1994-01-01   -2.218026    6.100382
1995-01-01   -1.923141    6.398158
